In [ ]:
# Import libraries
import gzip
import shutil
import os
import json 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Train Test Split, SMOTE
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# ML
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, accuracy_score, balanced_accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc,
    make_scorer, average_precision_score)
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from joblib import dump, load

# Reading and Parsing JSON

In [ ]:
# Specify the input and output file names (adjust file paths if needed)
input_file = '../data/dataset0.json.gz'  

# Unzip the file
def unzip_file(input):
    if input.endswith('.gz'):
        output = input[:-3]
    else:
        print("Input file is not a .gz file. Unzipping not required")
        return input
    
    with gzip.open(input, 'rb') as f_in:
        with open(output, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    print(f'File unzipped successfully. Output file: {output}')

    return output

In [ ]:
unzip_file(input_file)

In [ ]:
input_file = '../data/dataset0.json' 

def flatten_json(input_file):
    import json 
    import pandas as pd
    
    data = []
    with open(input_file, 'r') as file:
        for line in file:
            # Parse each line as a JSON object
            data.append(json.loads(line))
    
    rows = []

    for entry in data:
        for transcript_id, positions in entry.items():
            for position, sequences in positions.items():
                for sequence, features in sequences.items():
                    for feature_set in features:
                        row = {
                            'transcript_id': transcript_id,
                            'transcript_position': position,
                            'sequence': sequence,
                            '-1_dwelling_time': feature_set[0],
                            '-1_standard_dev': feature_set[1],
                            '-1_mean_current': feature_set[2],
                            '0_dwelling_time': feature_set[3],
                            '0_standard_dev': feature_set[4],
                            '0_mean_current': feature_set[5],
                            '+1_dwelling_time': feature_set[6],
                            '+1_standard_dev': feature_set[7],
                            '+1_mean_current': feature_set[8],
                        }
                        rows.append(row)

    df = pd.DataFrame(rows)
    return df

In [ ]:
# Flatten the data
flattened_data = flatten_json(input_file)
print(flattened_data.head())
print(flattened_data.info())

In [ ]:
# Export as csv
path = '../data/dataset0.csv'
flattened_data.to_csv(path, index=False)

# Load labels and features, perform aggregation

## Read in labels and features

In [ ]:
def read_labels(labels_file):

    labels = pd.read_csv(labels_file)
    return labels

labels = read_labels('../data/data.info.labelled')

print(labels.info())  # To get summary information about the DataFrame
print(labels.head())  # Preview the first few rows of the data

In [ ]:
features = pd.read_csv('../data/dataset0.csv')

print(features.info()) 
print(features.head())  

## Aggregate and label features

In [ ]:
def aggregate_by_transcript_position(features):

    # Apply mean, std, min, max, and skew to the selected columns
    features_agg = features.groupby(['transcript_id', 'transcript_position', 'sequence'])\
        .agg({
            '-1_dwelling_time': ['mean', 'min', 'max'],
            '-1_standard_dev': ['mean'],
            '-1_mean_current': ['mean', 'min', 'max'],
            '0_dwelling_time': ['mean', 'min', 'max'],
            '0_standard_dev': ['mean'],
            '0_mean_current': ['mean', 'min', 'max'],
            '+1_dwelling_time': ['mean', 'min', 'max'],
            '+1_standard_dev': ['mean'],
            '+1_mean_current': ['mean', 'min', 'max']
        }).reset_index()
    
    # Rename the columns to something more readable
    features_agg.columns = ['_'.join(col).strip() if col[1] else col[0] for col in features_agg.columns]

    

    return features_agg

In [ ]:
def one_hot_encode_sequence(features_agg, column='sequence'):
    # Step 1: Split each sequence into individual characters
    features_split = features_agg[column].apply(lambda x: pd.Series(list(x)))
    
    # Step 2: Remove the middle letter (always the 4th character, index 3 in 0-based index)
    features_split = features_split.drop(columns=[3])  # Drop the middle letter (index 3)
    
    # Step 3: One-hot encode the remaining letters
    # `pd.get_dummies` will automatically one-hot encode each position
    seq = pd.get_dummies(
        features_split,
        prefix=['pos1', 'pos2', 'pos3', 'pos5', 'pos6', 'pos7'],
        columns=[0, 1, 2, 4, 5, 6],
        dtype=int)

    expected_columns = [
        'pos1_A', 'pos1_T', 'pos1_C', 'pos1_G',
        'pos2_A', 'pos2_T', 'pos2_C', 'pos2_G',
        'pos3_A', 'pos3_T', 'pos3_C', 'pos3_G',
        'pos5_A', 'pos5_T', 'pos5_C', 'pos5_G',
        'pos6_A', 'pos6_T', 'pos6_C', 'pos6_G',
        'pos7_A', 'pos7_T', 'pos7_C', 'pos7_G'
    ]
    seq = seq.reindex(columns=expected_columns, fill_value=0)
    
    features_agg_with_seq = pd.concat([features_agg, seq], axis=1)
    features_agg_with_seq = features_agg_with_seq.drop(columns=[column])
    
    return features_agg_with_seq


In [ ]:
features_agg = aggregate_by_transcript_position(features)
features_agg_with_seq = one_hot_encode_sequence(features_agg)
print(features_agg_with_seq.info())
print(features_agg_with_seq.head())

In [ ]:
def add_gene_and_label(features, labels):
    """
    Adds gene_id and label to features dataframe
    
    Inputs:
    - features: pd.DataFrame
      Dataframe with selected features after feature engineering. Dataframe must contain transcript_id and transcript_position
    - labels: pd.DataFrame
      Dataframe with gene_id, transcript_id, transcript_position, and label.

    Output:
    - pd.DataFrame
      Updated features dataframe with added columns: gene_id and label from labels.
    """
    features_labelled = pd.merge(features, labels, on=['transcript_id', 'transcript_position'], how='inner')
    
    return features_labelled


In [ ]:
features_labelled = add_gene_and_label(features_agg_with_seq, labels)
print(features_labelled.info())

In [ ]:
# Breaking point: generate features_labelled.csv
features_labelled.to_csv('../data/features_labelled.csv',index=False)

# Train Test Split

In [ ]:
features_labelled = pd.read_csv('../data/features_labelled.csv')

In [ ]:
def get_X_y_id(features_labelled):

    y = features_labelled['label']
    id = features_labelled[['transcript_id','transcript_position']]
    X = features_labelled.drop(columns = ['transcript_id', 'transcript_position', 'gene_id', 'label'])
    return X, y, id

In [ ]:
def train_test_split_by_gene_id(features_labelled, features_columns):
    """
    Performs train test split based on gene_id. Returns X_train and X_test based on feature_columns
    
    Inputs:
    - features_labelled: pd.DataFrame
      Updated features dataframe with added columns: gene_id and label from labels.
      
    Output:
    - X_train: pd.DataFrame
    - X_test: pd.DataFrame
    - y_train: pd.DataFrame
    - y_test: pd.DataFrame
    """

    df = features_labelled

    # Get unique genes
    unique_genes = df['gene_id'].unique()
    
    # Perform the train-test split on genes
    genes_train, genes_test = train_test_split(unique_genes, test_size=0.2, random_state=42)
    
    # Split the dataset based on the gene split
    train_data = df[df['gene_id'].isin(genes_train)]
    test_data = df[df['gene_id'].isin(genes_test)]
    
    # Create the feature and target variables for training and testing
    id_train = train_data[['transcript_id','transcript_position']]
    X_train = train_data[features_columns]
    y_train = train_data['label']
    id_test = test_data[['transcript_id','transcript_position']]
    X_test = test_data[features_columns]
    y_test = test_data['label']
    
    # Output the shapes to verify the split
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Test Features Shape: {X_test.shape}")
    print(f"Test Labels Shape: {y_test.shape}")
    return (X_train, X_test, y_train, y_test, id_train, id_test)

In [ ]:
features_columns = [
        '-1_dwelling_time_mean', '-1_dwelling_time_min', '-1_dwelling_time_max',
        '-1_standard_dev_mean', 
        '-1_mean_current_mean', '-1_mean_current_min', '-1_mean_current_max',
        '0_dwelling_time_mean', '0_dwelling_time_min', '0_dwelling_time_max',
        '0_standard_dev_mean', 
        '0_mean_current_mean', '0_mean_current_min', '0_mean_current_max',
        '+1_dwelling_time_mean', '+1_dwelling_time_min', '+1_dwelling_time_max',
        '+1_standard_dev_mean', 
        '+1_mean_current_mean', '+1_mean_current_min', '+1_mean_current_max',
        'pos1_A', 'pos1_T', 'pos1_C', 'pos1_G',
        'pos2_A', 'pos2_T', 'pos2_C', 'pos2_G',
        'pos3_A', 'pos3_T', 'pos3_C', 'pos3_G',
        'pos5_A', 'pos5_T', 'pos5_C', 'pos5_G',
        'pos6_A', 'pos6_T', 'pos6_C', 'pos6_G',
        'pos7_A', 'pos7_T', 'pos7_C', 'pos7_G'
]
features_columns_means = [
    '-1_dwelling_time_mean',
    '-1_standard_dev_mean', 
    '-1_mean_current_mean',
    '0_dwelling_time_mean',
    '0_standard_dev_mean', 
    '0_mean_current_mean', 
    '+1_dwelling_time_mean',
    '+1_standard_dev_mean', 
    '+1_mean_current_mean'
]


In [ ]:
X_train_means_only, X_test_means_only, y_train, y_test, id_train, id_test = train_test_split_by_gene_id(features_labelled, features_columns_means)
X_train, X_test, y_train, y_test, id_train, id_test = train_test_split_by_gene_id(features_labelled, features_columns)

# Balancing Data

In [ ]:
def balance_train_data(X_train,y_train):
    """
    Performs SMOTE on train data, oversampling positive class, to account for imbalanced dataset
    
    Inputs:
    - X_train: pd.DataFrame
    - Y_train: pd.DataFrame
      
    Output:
    - X_train_resampled: pd.DataFrame
    - y_train_resampled: pd.DataFrame with balanced classes, ie the same number of 0s and 1s
    """

    from imblearn.over_sampling import SMOTE
    print(f'Label distribution before resampling:')
    print(pd.Series(y_train).value_counts())
    
    smote = SMOTE(k_neighbors=5, random_state=42) 
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    print(f'Label distribution after resampling:')
    print(pd.Series(y_train_resampled).value_counts())
    
    return X_train_resampled, y_train_resampled

# Models

## Predict and evaluate functions

In [ ]:
# Predict the output of xtest using the trained models
def predict(classifier, id, X):
    y_pred = classifier.predict(X)
    y_prob = classifier.predict_proba(X)[:, 1] #Gives probability estimates for y=1

    y_out = pd.DataFrame({
        'prediction': y_pred,                  
        'probability': y_prob          
    })

    result = pd.concat((id.reset_index(drop=True), y_out.reset_index(drop=True)), axis=1)
   
    # tabulate results
    prediction_counts = pd.Series(y_pred).value_counts().rename_axis('prediction').reset_index(name='count')
    print("\nTabulated Prediction Counts:")
    print(prediction_counts)

    return result

In [ ]:
# Model evaluation
def evaluate(y_test, predict_df):
    y_pred = predict_df['prediction']
    y_prob = predict_df['probability']
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test,  y_pred)
    balancedaccuracy = balanced_accuracy_score(y_test,  y_pred)
    f1score = f1_score(y_test,  y_pred) #F1 is a good scoring metric for imbalanced data when more attention is needed on the positives
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob) 
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_prob) # Computes ROC AUC score using probabilities of positive class
    pr_auc = auc(recall_vals
                 , precision_vals)

    print(f"Accuracy = {round(accuracy, ndigits=3)}")
    print(f"Balanced Accuracy = {round(balancedaccuracy, ndigits=3)}")
    print(f"f1 score = {round(f1score, ndigits=3)}")
    print(f"Precision = {round(precision, ndigits=3)}")
    print(f"Recall = {round(recall, ndigits=3)}")
    print(f"ROC AUC = {round(roc_auc, ndigits=3)}")
    print(f"PR AUC = {round(pr_auc, ndigits=3)}")

    # Plot confusion matrix
    plt.figure(figsize=(2,2))
    sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.figure(figsize=(5,5))  # Increase the figure size to avoid overlap
    plt.show()

    # Plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.show()

    # Plot Precision-Recall curve
    plt.plot(recall_vals, precision_vals, label='PR curve (AUC = %0.2f)' % pr_auc)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.show()

## Baseline Model - Logistic Regression with Means Only

In [ ]:
def LR(X_train,y_train,balance):
    if balance:
        X_train, y_train = balance_train_data(X_train, y_train)
    classifier = LogisticRegression(max_iter=1000, verbose = 1,  random_state = 123, class_weight = 'balanced')
    classifier.fit(X_train, y_train)
    return classifier

In [ ]:
LR_classifier_means = LR(X_train_means_only, y_train, False)
result = predict(LR_classifier_means, id_test, X_test_means_only)
evaluate(y_test, result)

## Model 1: Logistic Regression

In [ ]:
# Logistic Regression Model
def LR(X_train,y_train, balance):
    
    if balance:
        X_train, y_train = balance_train_data(X_train, y_train)
    # remove low variance features
    # Define the pipeline with variance threshold selector and logistic regression classifier
    model = Pipeline([
        ('selector', VarianceThreshold(threshold=0.01)),
        ('classifier', LogisticRegression(max_iter=1000, verbose=1, random_state=123, class_weight='balanced'))
    ])
    # Train the model on X_train and y_train
    model.fit(X_train, y_train)
    return model

In [ ]:
# ML workflow for Logistic Regression - without SMOTE
LR_classifier = LR(X_train, y_train, False)
result = predict(LR_classifier, id_test, X_test)
evaluate(y_test, result)

In [ ]:
# ML workflow for Logistic Regression - with SMOTE
LR_classifier_balanced = LR(X_train, y_train,True)
result = predict(LR_classifier, id_test, X_test)
evaluate(y_test, result)


In [ ]:
selection_mask= LR_classifier_balanced.named_steps['selector'].get_support()

# Create a DataFrame with feature names and selection status
feature_selection_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Selected': selection_mask
})

count_df = pd.Series(LR_classifier.named_steps['selector'].get_support()).value_counts().reset_index(name='Count').rename(columns={'index': 'Selected'})

print(feature_selection_df)
print(count_df)

## Model 2: Random Forest

### Baseline and Selector

In [ ]:
def RF(X_train, y_train):
    classifier = RandomForestClassifier(n_estimators=100, random_state=123, class_weight='balanced', verbose=1)
    classifier.fit(X_train, y_train)
    return classifier

In [ ]:
RF_classifier = RF(X_train, y_train)
result = predict(RF_classifier, id_test, X_train)
evaluate(y_train, result)
result = predict(RF_classifier, id_test, X_test)
evaluate(y_test, result)

In [ ]:
def RF_tuning_feature_selection_RF(X_train, y_train):

    '''
    RF_tuning_feature_selection_RF uses SelectFromModel to select features with higher feature importance than median.
    This step was done before tuning due to the large number of features and high complexity of dataset to reduce training time.

    Inputs:
    - X_train
    - y_train

    Output:
    - selector_model: This model should be applied on X_train before further RF tuning. 
    
    '''

    # Step 1: Initialize RandomForestClassifier and fit for feature selection
    rf_temp = RandomForestClassifier(random_state=123, class_weight='balanced', n_estimators=50)
    selector_model = SelectFromModel(rf_temp, threshold="median")
    
    # Fit the selector model on training data
    selector_model.fit(X_train, y_train)
    
    # Get the boolean array indicating which features were kept
    feature_support = selector_model.get_support()
    
    # Get feature names from X_train (assuming it’s a DataFrame), otherwise create generic names
    feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'feature_{i}' for i in range(X_train.shape[1])]
    
    # Retrieve feature importances from the fitted RandomForestClassifier model
    feature_importances = selector_model.estimator_.feature_importances_
    
    # Create the DataFrame to show feature selection status and importance
    feature_df = pd.DataFrame({
        'Feature': feature_names,
        'Kept': feature_support,
        'Importance': feature_importances
    })

    feature_df = feature_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)
    
    # Display the DataFrame
    print(feature_df)
    
    return selector_model



In [ ]:
selector = RF_tuning_feature_selection_RF(X_train, y_train)

In [ ]:
# After defining RF_classifier
dump(selector, '../model/selector.joblib')  # Save the model

In [ ]:
def select_features(X_train, selector):
    X_train = selector.transform(X_train)
    
    return X_train

### Tuning

In [ ]:
def RF_tuning_intermediate(X_train, y_train, param_dist, selector):

    '''
    RF_tuning_intermediate uses RandomizedSearch to reduce search space
    with feature selection to improve efficiency. It also provides a
    DataFrame indicating which features were kept or removed.
    '''

    X_train = select_features(X_train, selector)
    
    # Step 1: Define the main RandomForestClassifier for tuning
    model = RandomForestClassifier(random_state=123, class_weight='balanced')

    # Custom combined AUC score
    def combined_auc_score(y_true, y_pred_proba):
        roc_auc = roc_auc_score(y_true, y_pred_proba)
        pr_auc = average_precision_score(y_true, y_pred_proba)
        return 0.3 * roc_auc + 0.7 * pr_auc

    # Create custom scorer
    custom_scorer = make_scorer(combined_auc_score, needs_proba=True)

    # Step 2: Set up RandomizedSearchCV
    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=20,  
        scoring=custom_scorer,
        cv=5,      
        n_jobs=-1,
        verbose=1,
        random_state=123
    )
    
    # Train the model using RandomizedSearchCV
    randomized_search.fit(X_train, y_train)
    
    return randomized_search.best_estimator_, randomized_search.best_params_

In [ ]:
param_dist_search_1 = {
            'n_estimators': [100, 200, 300],  
            'max_depth': [None, 10, 20, 30],  
            'min_samples_split': [2, 3], 
            'min_samples_leaf': [1, 2, 3]
        }    
RF_search_1_classifier, RF_search_1_params = RF_tuning_intermediate(X_train, y_train, param_dist_search_1, selector)
print(f'Tuning step 1 outputs: {RF_search_1_params}')

In [ ]:
# Results from tuning 1: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': None}
# Do tuning 2 based on tuning 1 results
param_dist_search_2 = {
            'n_estimators': [300, 400],  # Tuning 1 findings: 300 > 100/200
            'max_depth': [None, 40, 50], # Tuning 1 findings: None > 10/20/30
            'min_samples_split': [2], # Tuning 1 findings: 2 > 3
            'min_samples_leaf': [3, 4],  # Tuning 1 findings: 3 > 1/2
            'max_features': ['sqrt', 'log2'],  # New feat in tuning 2
            'bootstrap': [True, False]  # New feat in tuning 2
        }    
RF_search_2_classifier, RF_search_2_params = RF_tuning_intermediate(X_train, y_train, param_dist_search_2, selector)
print(f'Tuning step 2 outputs: {RF_search_2_params}')

In [ ]:
# Tuning 2 results - train
X_train_selected = select_features(X_train,selector)
result = predict(RF_search_2_classifier, id_train, X_train_selected)
evaluate(y_train, result)

# noticed tendency to overfit - prioritise reducing complexity

In [ ]:
# Results from tuning 2: {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}
param_dist_search_3 = {
            'n_estimators': [100, 200, 300, 400],  
            'max_depth': [10, 20, 30], # decrease to reduce complexity 
            'max_leaf_nodes': [10, 30, 50, 70, 100], # decrease to reduce complexity
            'min_samples_split': [5, 10, 15, 20], # increase to reduce complexity
            'min_samples_leaf': [2, 5, 10] # increase to reduce complexity
        }    
RF_search_3_classifier, RF_search_3_params = RF_tuning_intermediate(X_train, y_train, param_dist_search_3, selector)
print(f'Tuning step 3 outputs: {RF_search_3_params}')

In [ ]:
# Tuning 3 results - train
X_train_selected = select_features(X_train,selector)
result = predict(RF_search_3_classifier, id_train, X_train_selected)
evaluate(y_train, result)
# underfitting train data, increase complexity

In [ ]:
# Results from tuning 3: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 10, 'max_leaf_nodes': 70, 'max_depth': 10}
param_dist_search_4 = {
            'n_estimators': [150, 200, 250], 
            'max_depth': [10], # keep as lowest number was most optimal
            'max_leaf_nodes': [75, 100, 125], # can be increased to prevent underfitting
            'min_samples_split': [3, 4, 5], # can be decreased to prevent underfiting
            'min_samples_leaf': [10] # keep as highest number was most optimal
        }    
RF_search_4_classifier, RF_search_4_params = RF_tuning_intermediate(X_train, y_train, param_dist_search_4, selector)
print(f'Tuning step 4 outputs: {RF_search_4_params}')

In [ ]:
# Tuning 4 results - train
X_train_selected = select_features(X_train,selector)
result = predict(RF_search_4_classifier, id_train, X_train_selected)
evaluate(y_train, result)


In [ ]:
# Tuning 4 results - test
X_test_selected = select_features(X_test,selector)
result = predict(RF_search_4_classifier, id_test, X_test_selected)
evaluate(y_test, result)

In [ ]:
# Results from tuning 4:  {'n_estimators': 250, 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_leaf_nodes': 125, 'max_depth': 10}
param_dist_search_5 = {
            'n_estimators': [250, 500, 750, 1000], 
            'max_depth': [10], # keep as lowest number was most optimal
            'max_leaf_nodes': [125, 250, 500, 750, 1000], # can be increased to prevent underfitting
            'min_samples_split': [4], # keep as middle number was selected
            'min_samples_leaf': [8, 10], 
        }   

RF_search_5_classifier, RF_search_5_params = RF_tuning_intermediate(X_train, y_train, param_dist_search_5, selector)
print(f'Tuning step 5 outputs: {RF_search_5_params}')

In [ ]:
# Tuning step 5 outputs: {'n_estimators': 500, 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_leaf_nodes': 1000, 'max_depth': 10}
# Tuning 5 results - train
X_train_selected = select_features(X_train,selector)
result = predict(RF_search_5_classifier, id_train, X_train_selected)
evaluate(y_train, result)

In [ ]:
# Tuning 5 results - test
X_test_selected = select_features(X_test,selector)
result = predict(RF_search_5_classifier, id_test, X_test_selected)
evaluate(y_test, result)

In [ ]:
def RF_tuning_final(X_train, y_train, param_grid, selector):
    '''
    RF_tuning uses GridSearch on an exhaustive list of parameters, determined after GridSearch
    '''   
    X_train = select_features(X_train, selector)

    # Define the random forest model with class_weight set to 'balanced'
    model = RandomForestClassifier(random_state=123, class_weight='balanced')

    # Custom scoring function combining ROC AUC and PR AUC
    def combined_auc_score(y_true, y_pred_proba):
        roc_auc = roc_auc_score(y_true, y_pred_proba)
        pr_auc = average_precision_score(y_true, y_pred_proba)
    
        # Combined score
        combined_score = 0.3 * roc_auc + 0.7 * pr_auc # prioritise pr_auc due to imbalanced data
        return combined_score

    # Create a custom scorer using make_scorer
    custom_scorer = make_scorer(combined_auc_score, needs_proba=True)

    # Set up GridSearchCV with the parameter grid
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                               scoring=custom_scorer, cv=5, n_jobs=-1, verbose=1)
    
    # Train the model on X_train and y_train using GridSearchCV
    grid_search.fit(X_train, y_train)

    return grid_search.best_estimator_, grid_search.best_params_

In [ ]:
# step 5 outputs: {'n_estimators': 500, 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_leaf_nodes': 1000, 'max_depth': 10}
param_grid_final = {
            'n_estimators': [500], 
            'max_depth': [10],
            'max_leaf_nodes': [1000, 1500, 2000], # can be increased to prevent underfitting
            'min_samples_split': [4], 
            'min_samples_leaf': [10], 
            'bootstrap': [True, False], # new
            'max_features': ['sqrt', 'log2'] # new
        }   
RF_tuned_classifier, RF_tuned_params = RF_tuning_final(X_train, y_train, param_grid_final, selector)
print(f'Final classifier: {RF_tuned_params}')

In [ ]:
# Final classifier: {'bootstrap': False, 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': 1000, 'min_samples_leaf': 10, 'min_samples_split': 4, 'n_estimators': 500}

In [ ]:
# Final train results
X_train_selected = select_features(X_train, selector)
result = predict(RF_tuned_classifier, id_test, X_train_selected)
evaluate(y_train, result)

In [ ]:
# Final test results
X_test = select_features(X_test, selector)
result = predict(RF_tuned_classifier, id_test, X_test)
evaluate(y_test, result)

# Final Model - Random Forest: Train using all the data

In [ ]:
X_full = features_labelled.drop(columns = ['gene_id','transcript_id','transcript_position','label'])
X_selected = select_features(X_full, selector)
y_full = features_labelled['label']

RF_final_classifier = RandomForestClassifier(**RF_tuned_params, random_state=123, class_weight='balanced') #use ideal params
RF_final_classifier.fit(X_selected, y_full)

In [ ]:
# After defining RF_classifier
dump(RF_final_classifier, '../model/rf_classifier.joblib')  # Save the model

In [ ]:
X, y, id = get_X_y_id(features_labelled)
X_all = select_features(X, selector)
result = predict(RF_final_classifier, id, X_all)
evaluate(y, result)

# Run Predictions

In [ ]:
def load_model(job_lib_file): 

    # Load the model
    classifier = load(job_lib_file)

    return classifier

In [ ]:
def split_id_cols(X_with_id):
    '''
    Remove specific ID columns from the DataFrame.

    Parameters:
    X_all (pd.DataFrame): The input DataFrame from which the specified columns will be removed.

    Returns:
    pd.DataFrame: A DataFrame with the specified ID columns removed. If the columns do not exist, the original DataFrame is returned unchanged.
    
    Columns to be removed:
    - 'transcript_id'
    - 'transcript_position'
    '''

    # Drop specified columns from the DataFrame
    id = X_with_id[['transcript_id', 'transcript_position']]
    X = X_with_id.drop(columns=['transcript_id', 'transcript_position'], errors='ignore')
    return X, id

def clean_result_format(result):
    '''
    Converts results to desired results format
    '''
    result = result.rename(columns={'probability': 'score'})
    result = result[['transcript_id', 'transcript_position', 'score']]
    return result

def generate_predictions(json_gz_file, selector_file, classifier_file, output_path='default', include_features=False):
    '''
    Generates predictions on test dataset given json_gz_file and classifier_file

    Parameters:
    - json_gz_file
    - classifier_file

    Returns:
    - pd.Dataframe: output of predictions with transcript_id, and also outputs csv
    '''
    
    json_file = unzip_file(json_gz_file)
    features = flatten_json(json_file)
    print("Step 1/6: Processing JSON completed.")
    
    features_agg = aggregate_by_transcript_position(features)
    features_agg_with_seq = one_hot_encode_sequence(features_agg)
    X, id = split_id_cols(features_agg_with_seq)
    print("Step 2/6: Cleaning features completed.")
    
    selector = load_model(selector_file)
    X_selected = select_features(X, selector)
    print("Step 3/6: Feature selection completed.")
    
    classifier = load_model(classifier_file)
    result = predict(classifier, id, X_selected)
    print("Step 4/6: Prediction using model completed.")
   
    result = clean_result_format(result)
    print(result.head())
    print("Step 5/6: Cleaning results completed.")

    if include_features:
        result = pd.concat([result, X], axis=1)

    if output_path == 'default':
        output_path = json_file.replace('.json', '.csv').replace('/data/', '/output/')
        
    result.to_csv(output_path, index=False)  # Write to CSV without the index
    print(f"Step 6/6: Results written to {output_path}")

In [ ]:
# dataset0 predictions
generate_predictions('../data/dataset0.json.gz', '../model/selector.joblib.gz', '../model/rf_classifier.joblib.gz')

In [ ]:
# dataset1 predictions
generate_predictions('../data/dataset1.json.gz', '../model/selector.joblib.gz', '../model/rf_classifier.joblib.gz')

In [ ]:
# dataset2 predictions
generate_predictions('../data/dataset2.json.gz', '../model/selector.joblib.gz', '../model/rf_classifier.joblib.gz')

In [ ]:
# dataset3 predictions
generate_predictions('../data/dataset3.json.gz', '../model/selector.joblib.gz', '../model/rf_classifier.joblib.gz')